# 1. CLUSTER (MICRO + MACRO) + SUPABASE SAVE

## ONE RUN

In [ ]:
# ========== 1. Imports ==========
import json
import psycopg2
import pandas as pd
import numpy as np
import ast
import umap
import hdbscan
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from psycopg2.extras import execute_values

# ========== 2. Supabase DB Connection ==========
SUPABASE_DB_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"
SUPABASE_DB_NAME = "postgres"
SUPABASE_DB_USER = "postgres.kxuzwqoeocgvwamamqbx"
SUPABASE_DB_PASS = "SEO_seo_530"
SUPABASE_DB_PORT = "6543"

conn = psycopg2.connect(
    host=SUPABASE_DB_HOST,
    dbname=SUPABASE_DB_NAME,
    user=SUPABASE_DB_USER,
    password=SUPABASE_DB_PASS,
    port=SUPABASE_DB_PORT,
    sslmode="require"
)
cur = conn.cursor()
print("✅ Connected to Supabase")

# ========== 3. Fetch Document + Embeddings ==========
query = """
SELECT v.id,
       v.suggested_title AS suggested_title_embedding,
       v.metadata AS metadata_embedding,
       v.user_search_intent AS user_search_intent_embedding,
       v.faq_pairs AS faq_pairs_embedding,
       v.entities AS entities_embedding,
       d.url,
       d.suggested_title AS suggested_title_text,
       d.user_search_intent AS user_search_intent_text,
       d.metadata_h1,
       d.metadata_h2,
       d.metadata_h3,
       d.faq_pairs AS faq_pairs_text,
       d.entities AS entities_text
FROM vecs.data_vector v
JOIN public.data_text d ON d.id = v.id
"""
df = pd.read_sql(query, conn)
print(f"✅ Data fetched: {len(df)} documents")

# ========== 4. Convert embeddings ==========
def parse_embedding(x):
    if x is None:
        return np.zeros(768, dtype=np.float32)
    try:
        return np.array(ast.literal_eval(x), dtype=np.float32)
    except Exception:
        return np.zeros(768, dtype=np.float32)

for col in ["suggested_title_embedding", "metadata_embedding",
            "user_search_intent_embedding", "faq_pairs_embedding",
            "entities_embedding"]:
    df[col] = df[col].apply(parse_embedding)

# Combine embeddings
df["combined_embedding"] = df.apply(
    lambda row: np.mean([
        row["suggested_title_embedding"],
        row["metadata_embedding"],
        row["user_search_intent_embedding"],
        row["faq_pairs_embedding"],
        row["entities_embedding"]
    ], axis=0),
    axis=1
)

# =========================
# 5. Macro Clustering
# =========================
X = np.vstack(df['combined_embedding'].values)

umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=50,
    metric='cosine',
    random_state=42
)
X_umap = umap_model.fit_transform(X)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=30,
    metric='euclidean',
    cluster_selection_method='eom'
)
macro_labels = clusterer.fit_predict(X_umap)
df['macro_cluster_id'] = macro_labels

# ✅ Evaluate Macro Clusters
mask = macro_labels != -1
if mask.sum() > 1 and len(set(macro_labels[mask])) > 1:
    sil = silhouette_score(X_umap[mask], macro_labels[mask])
    dbi = davies_bouldin_score(X_umap[mask], macro_labels[mask])
    chi = calinski_harabasz_score(X_umap[mask], macro_labels[mask])
    print(f"[Macro] Silhouette: {sil:.4f}, Davies-Bouldin: {dbi:.4f}, Calinski-Harabasz: {chi:.4f}")
else:
    print("[Macro] Not enough clusters for evaluation")

print(f"[Macro] Number of clusters: {len(set(macro_labels)) - (1 if -1 in macro_labels else 0)}")
print(f"[Macro] Number of noise points: {sum(macro_labels == -1)}")

# =========================
# 6. Micro Clustering
# =========================
def run_micro_clustering(subset, min_cluster_size=5, n_neighbors=10, n_components=10):
    """Run UMAP + HDBSCAN on a subset and return labels + reduced embeddings"""
    X_sub = np.vstack(subset["combined_embedding"].values)

    n_comp = min(n_components, X_sub.shape[0] - 1)
    n_neigh = min(n_neighbors, X_sub.shape[0] - 1)

    umap_sub = umap.UMAP(
        n_neighbors=n_neigh,
        n_components=n_comp,
        metric="cosine",
        random_state=42
    )
    X_sub_umap = umap_sub.fit_transform(X_sub)

    micro_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric="euclidean",
        cluster_selection_method="eom"
    )
    micro_labels = micro_clusterer.fit_predict(X_sub_umap)
    return micro_labels, X_sub_umap

df["micro_cluster_id"] = -1
sil_threshold = 0.6

for cluster_id in set(macro_labels):
    if cluster_id == -1:
        continue

    subset = df[df["macro_cluster_id"] == cluster_id]
    if len(subset) < 10:
        continue

    # First attempt
    micro_labels, X_sub_umap = run_micro_clustering(subset, min_cluster_size=3, n_neighbors=5, n_components=5)
    df.loc[subset.index, "micro_cluster_id"] = micro_labels

    # Evaluate
    mask = micro_labels != -1
    if mask.sum() > 1 and len(set(micro_labels[mask])) > 1:
        sil = silhouette_score(X_sub_umap[mask], micro_labels[mask])
        dbi = davies_bouldin_score(X_sub_umap[mask], micro_labels[mask])
        chi = calinski_harabasz_score(X_sub_umap[mask], micro_labels[mask])
        print(f"[Micro | Macro {cluster_id}] Silhouette: {sil:.4f}, DBI: {dbi:.4f}, CHI: {chi:.4f}")

        # 🔄 Retry if silhouette too low
        if sil < sil_threshold and sil != -1:
            print(f"[Recluster] Macro {cluster_id} → retry with different hyperparameters...")
            micro_labels, X_sub_umap = run_micro_clustering(subset, min_cluster_size=2, n_neighbors=5, n_components=1)
            df.loc[subset.index, "micro_cluster_id"] = micro_labels

            # Re-evaluate
            mask = micro_labels != -1
            if mask.sum() > 1 and len(set(micro_labels[mask])) > 1:
                sil = silhouette_score(X_sub_umap[mask], micro_labels[mask])
                dbi = davies_bouldin_score(X_sub_umap[mask], micro_labels[mask])
                chi = calinski_harabasz_score(X_sub_umap[mask], micro_labels[mask])
                print(f"[Re-Micro | Macro {cluster_id}] Silhouette: {sil:.4f}, DBI: {dbi:.4f}, CHI: {chi:.4f}")
            else:
                print(f"[Re-Micro | Macro {cluster_id}] Still not enough clusters.")
    else:
        print(f"[Micro | Macro {cluster_id}] Not enough clusters for evaluation")

# =========================
# 7. Save Cluster Metadata
# =========================
macro_info = []
micro_info = []
assignments = []

for cluster_id in set(macro_labels):
    if cluster_id == -1:
        continue

    cluster_embeddings = np.vstack(df[df['macro_cluster_id'] == cluster_id]['combined_embedding'].values)
    centroid = cluster_embeddings.mean(axis=0).tolist()
    count = len(cluster_embeddings)

    macro_info.append({
        "cluster_id": int(cluster_id),
        "count": count,
        "cluster_name": "",   # empty
        "representative_text": "",  # empty
        "representative_keywords": "",  # empty
        "centroid_embedding": json.dumps(centroid)
    })

    # Micro clusters inside macro
    subset = df[df["macro_cluster_id"] == cluster_id]
    for mc_id in set(subset["micro_cluster_id"]):
        if mc_id == -1:
            continue
        mc_embeddings = np.vstack(subset[subset["micro_cluster_id"] == mc_id]['combined_embedding'].values)
        mc_centroid = mc_embeddings.mean(axis=0).tolist()
        micro_info.append({
            "cluster_id": int(cluster_id),
            "microcluster_id": int(mc_id),
            "microcluster_name": "",  # empty
            "count": len(mc_embeddings),
            "representative_text": "",  # empty
            "representative_keywords": "",  # empty
            "centroid_embedding": json.dumps(mc_centroid)
        })

# Assignments
for _, row in df.iterrows():
    assignments.append({
        "doc_id": str(row["id"]),
        "cluster_id": int(row["macro_cluster_id"]),
        "microcluster_id": int(row["micro_cluster_id"]),
        "probability": 1.0  # no probability in HDBSCAN like BERTopic, set default 1.0
    })

# =========================
# 8. Save to Supabase
# =========================
print("\n🚀 Inserting results into Supabase...")

cur.execute("DROP TABLE IF EXISTS cluster_assignments")
cur.execute("DROP TABLE IF EXISTS cluster_macro")
cur.execute("DROP TABLE IF EXISTS cluster_micro")

cur.execute("""
    CREATE TABLE cluster_assignments (
        doc_id UUID,
        cluster_id INT,
        microcluster_id INT,
        probability FLOAT
    )
""")

cur.execute("""
    CREATE TABLE cluster_macro (
        cluster_id INT PRIMARY KEY,
        count INT,
        cluster_name TEXT,
        representative_text TEXT,
        representative_keywords TEXT,
        centroid_embedding JSON
    )
""")

cur.execute("""
    CREATE TABLE cluster_micro (
        cluster_id INT,
        microcluster_id INT,
        microcluster_name TEXT,
        count INT,
        representative_text TEXT,
        representative_keywords TEXT,
        centroid_embedding JSON
    )
""")

# Insert Macro
macro_values = [
    (row["cluster_id"], row["count"], row["cluster_name"], row["representative_text"],
     row["representative_keywords"], row["centroid_embedding"])
    for row in macro_info
]
execute_values(cur, """
    INSERT INTO cluster_macro (cluster_id, count, cluster_name, representative_text, representative_keywords, centroid_embedding)
    VALUES %s
""", macro_values)

# Insert Micro
micro_values = [
    (row["cluster_id"], row["microcluster_id"], row["microcluster_name"], row["count"],
     row["representative_text"], row["representative_keywords"], row["centroid_embedding"])
    for row in micro_info
]
execute_values(cur, """
    INSERT INTO cluster_micro (cluster_id, microcluster_id, microcluster_name, count, representative_text, representative_keywords, centroid_embedding)
    VALUES %s
""", micro_values)

# Insert Assignments
assign_values = [
    (row["doc_id"], row["cluster_id"], row["microcluster_id"], row["probability"])
    for row in assignments
]
execute_values(cur, """
    INSERT INTO cluster_assignments (doc_id, cluster_id, microcluster_id, probability)
    VALUES %s
""", assign_values, page_size=1000)

conn.commit()
cur.close()
conn.close()

print("✅ All results saved to Supabase!")
print("\n🎉 Pipeline completed successfully!")


✅ Connected to Supabase


/tmp/ipython-input-3642317363.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ Data fetched: 12058 documents


/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[Macro] Silhouette: 0.6536, Davies-Bouldin: 0.3876, Calinski-Harabasz: 11844.8389
[Macro] Number of clusters: 69
[Macro] Number of noise points: 986


...

[Micro | Macro 63] Silhouette: 0.6300, DBI: 0.4462, CHI: 1056.3746
[Micro | Macro 64] Silhouette: 0.8182, DBI: 0.2113, CHI: 3863.1289


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/umap/umap_.py:

[Micro | Macro 65] Silhouette: 0.8652, DBI: 0.1996, CHI: 794.0573
[Micro | Macro 66] Silhouette: 0.7977, DBI: 0.1942, CHI: 754.5507
[Micro | Macro 67] Silhouette: 0.9069, DBI: 0.1245, CHI: 2378.8359


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[Micro | Macro 68] Silhouette: 0.7354, DBI: 0.3346, CHI: 730.7753

🚀 Inserting results into Supabase...
✅ All results saved to Supabase!

🎉 Pipeline completed successfully!


# 2. TEXT : MICROCLUSTERS

## ONE RUN

In [ ]:
!pip install supabase

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import ast
import math
import json
from typing import Dict, Any
from openai import OpenAI
from supabase import create_client, Client

# =====================================================
# --- CONFIG ---
# =====================================================
# Supabase (DB + API)
SUPABASE_DB_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"
SUPABASE_DB_NAME = "postgres"
SUPABASE_DB_USER = "postgres.kxuzwqoeocgvwamamqbx"
SUPABASE_DB_PASS = "SEO_seo_530"
SUPABASE_DB_PORT = "6543"

OPENAI_API_KEY = "...."
SUPABASE_URL = "...."
SUPABASE_KEY = "..."

# --- Target selection (only change here) ---
TARGET_CLUSTER_ID = 7
TARGET_MICROCLUSTER_ID = 17
MICRO_ASSIGN_COL = "microcluster_id"

# =====================================================
# --- DB CONNECTION ---
# =====================================================
conn = psycopg2.connect(
    host=SUPABASE_DB_HOST,
    dbname=SUPABASE_DB_NAME,
    user=SUPABASE_DB_USER,
    password=SUPABASE_DB_PASS,
    port=SUPABASE_DB_PORT,
    sslmode="require"
)
print("✅ DB connected")

# =====================================================
# --- LOAD DATA ---
# =====================================================
def parse_embedding(x):
    if x is None:
        return np.zeros(768, dtype=np.float32)
    try:
        return np.array(ast.literal_eval(x), dtype=np.float32)
    except Exception:
        return np.zeros(768, dtype=np.float32)

base_sql = f"""
SELECT
  ca.doc_id AS id,
  d.suggested_title        AS suggested_title_text,
  d.user_search_intent     AS user_search_intent_text,
  d.entities               AS entities_text,
  v.suggested_title        AS suggested_title_embedding,
  v.metadata               AS metadata_embedding,
  v.user_search_intent     AS user_search_intent_embedding,
  v.faq_pairs              AS faq_pairs_embedding,
  v.entities               AS entities_embedding
FROM public.cluster_assignments ca
JOIN public.data_text d     ON d.id = ca.doc_id
JOIN vecs.data_vector v     ON v.id = ca.doc_id
WHERE ca.cluster_id = %s
"""

params = [TARGET_CLUSTER_ID]
if TARGET_MICROCLUSTER_ID is not None:
    base_sql += f" AND ca.{MICRO_ASSIGN_COL} = %s"
    params.append(TARGET_MICROCLUSTER_ID)

df = pd.read_sql(base_sql, conn, params=params)
print(f"✅ Loaded {len(df)} docs for cluster={TARGET_CLUSTER_ID}, micro={TARGET_MICROCLUSTER_ID}")

for col in [
    "suggested_title_embedding",
    "metadata_embedding",
    "user_search_intent_embedding",
    "faq_pairs_embedding",
    "entities_embedding",
]:
    df[col] = df[col].apply(parse_embedding)

df["combined_embedding"] = df.apply(
    lambda r: np.mean(
        [
            r["suggested_title_embedding"],
            r["metadata_embedding"],
            r["user_search_intent_embedding"],
            r["faq_pairs_embedding"],
            r["entities_embedding"],
        ],
        axis=0,
    ),
    axis=1,
)

df["entities_text"] = df["entities_text"].apply(lambda x: str(x) if x is not None else "")

print("🔹 Preview:")
print(df.head())

# =====================================================
# --- PROMPT BUILDER ---
# =====================================================
def build_micro_prompt(df_chunk: pd.DataFrame, cluster_id: int, micro_id: Any) -> str:
    docs = []
    for _, row in df_chunk.iterrows():
        title = row.get('suggested_title_text', '') or ""
        intent = row.get('user_search_intent_text', '') or ""
        entities = row.get('entities_text', '') or ""
        doc_str = f"- Title: {title}\n  Intent: {intent}\n  Entities: {entities}\n"
        docs.append(doc_str)
    joined_docs = "\n".join(docs)

    return f"""
You are analyzing a microcluster of documents. Each document contains several fields
(title, search intent, entities, etc.).

Cluster ID: {cluster_id}, Microcluster ID: {micro_id}

Here are the documents from this microcluster:
---
{joined_docs}
---

Please create a structured JSON object with the following fields:

{{
  "summary": "A 2–3 sentence description of the main theme of this microcluster.",
  "key_points": ["3–6 bullet points highlighting key recurring ideas or topics."],
  "keywords": ["10–20 representative keywords or phrases (short, lowercase)."],
  "microcluster_name": "A short 2–5 word descriptive label for this microcluster."
}}

Rules:
- Do not invent facts not present in the documents.
- Use clear, concise English.
- Keywords should reflect actual terms from the text.
- The microcluster name should be broad enough to describe the set, but specific enough to distinguish it.
""".strip()

# =====================================================
# --- SUMMARIZATION STRATEGY ---
# =====================================================
def summarize_microcluster(df_subset: pd.DataFrame, cluster_id: int, micro_id: Any,
                           batch_size: int = 25) -> Dict[str, Any]:
    result = {"mode": None, "prompts": [], "info": {}}
    n = len(df_subset)
    if n == 0:
        result["mode"] = "empty"
        result["info"] = {"n_docs": 0}
        return result

    df_local = df_subset.copy().reset_index(drop=False)
    df_local["__emb"] = df_local["combined_embedding"].apply(lambda x: np.asarray(x, dtype=float))
    result["info"]["n_docs"] = n

    if n < 10:
        result["mode"] = "small"
        result["prompts"].append({
            "prompt_id": f"{micro_id}_all",
            "prompt": build_micro_prompt(df_local, cluster_id, micro_id),
            "doc_indices": df_local["index"].tolist(),
            "doc_ids": df_local["id"].tolist()
        })
        result["info"]["selection"] = "all_docs"

    elif n <= 30:
        result["mode"] = "medium"
        emb_stack = np.vstack(df_local["__emb"].values)
        centroid = emb_stack.mean(axis=0)
        df_local["__dist"] = df_local["__emb"].apply(lambda v: np.linalg.norm(v - centroid))
        top_df = df_local.nsmallest(10, "__dist")
        result["prompts"].append({
            "prompt_id": f"{micro_id}_top10",
            "prompt": build_micro_prompt(top_df, cluster_id, micro_id),
            "doc_indices": top_df["index"].tolist(),
            "doc_ids": top_df["id"].tolist()
        })
        result["info"]["selection"] = "topk_centroid"

    else:
        result["mode"] = "large"
        batches = np.array_split(df_local, math.ceil(n / batch_size))
        for idx, batch in enumerate(batches, start=1):
            result["prompts"].append({
                "prompt_id": f"{micro_id}_part{idx}",
                "prompt": build_micro_prompt(batch, cluster_id, f"{micro_id}_part{idx}"),
                "doc_indices": batch["index"].tolist(),
                "doc_ids": batch["id"].tolist()
            })
        result["info"]["selection"] = "batches"

    return result

# =====================================================
# --- LLM + SUPABASE ---
# =====================================================
client = OpenAI(api_key=OPENAI_API_KEY)
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

def run_llm(prompt: str) -> dict:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that outputs only valid JSON."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.3,
    )
    raw_text = response.choices[0].message.content.strip()
    try:
        return json.loads(raw_text)
    except json.JSONDecodeError:
        print("⚠️ Invalid JSON from LLM, returning raw text")
        return {"raw": raw_text}

def save_microcluster_result(cluster_id, microcluster_id, llm_result: dict):
    summary = llm_result.get("summary", "")
    key_points = llm_result.get("key_points", [])
    keywords = llm_result.get("keywords", [])
    micro_name = llm_result.get("microcluster_name", "")

    if isinstance(key_points, list):
        key_points = "; ".join([str(k) for k in key_points])
    if isinstance(keywords, list):
        keywords = ", ".join([str(k) for k in keywords])

    representative_text = summary
    if key_points:
        representative_text += "\n\n- " + "\n- ".join(key_points.split("; "))

    data = {
        "cluster_id": cluster_id,
        "microcluster_id": microcluster_id,
        "microcluster_name": micro_name,
        "representative_text": representative_text,
        "representative_keywords": keywords,
    }

    result = supabase.table("cluster_micro") \
                     .update(data) \
                     .eq("cluster_id", cluster_id) \
                     .eq("microcluster_id", microcluster_id) \
                     .execute()
    print("✅ Saved to Supabase:", result)

# =====================================================
# --- MAIN RUN ---
# =====================================================
res = summarize_microcluster(df, cluster_id=TARGET_CLUSTER_ID, micro_id=TARGET_MICROCLUSTER_ID)

for p in res["prompts"]:
    print(f"⚡ Sending prompt {p['prompt_id']} to LLM...")
    llm_result = run_llm(p["prompt"])
    print("🔎 LLM Result:", llm_result)
    save_microcluster_result(TARGET_CLUSTER_ID, TARGET_MICROCLUSTER_ID, llm_result)


✅ DB connected


/tmp/ipython-input-246136831.py:76: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(base_sql, conn, params=params)


✅ Loaded 3 docs for cluster=7, micro=17
🔹 Preview:
                                     id  \
0  43698fc0-29f6-47dd-ae4b-e085ee0be3ac   
1  8a0ff72a-3858-40e7-8eab-844c93eec171   
2  8461e637-e851-40d1-b3b6-3ee7ca1ba5e1   

                                suggested_title_text  \
0  Review del Infiniti II I CV: Características, ...   
1  Review del Infiniti M III I CV: Característica...   
2  Reseña del Invicta I CV: Características, Prec...   

                             user_search_intent_text  \
0  Información sobre el Infiniti II I CV, incluye...   
1  Buscar información sobre el Infiniti M III I C...   
2  Informarse sobre el producto Invicta I CV, sus...   

                                       entities_text  \
0  {'media': ['foto infiniti ii i cv', 'vídeo des...   
1  {'foto': ['infiniti m iii i cv'], 'precio': ['...   
2  {'media': ['foto', 'vídeo destacado', 'noticia...   

                           suggested_title_embedding  \
0  [-1.2643936, 1.0081645, -0.75931627, 0.579

# 3. TEXT : MACROCLUSTER

## ONE RUN

In [ ]:
import json
import math
from typing import Dict, Any
from openai import OpenAI
from supabase import create_client, Client

# =====================================================
# --- CONFIG ---
# =====================================================
# Supabase (DB + API)
SUPABASE_DB_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"
SUPABASE_DB_NAME = "postgres"
SUPABASE_DB_USER = "postgres.kxuzwqoeocgvwamamqbx"
SUPABASE_DB_PASS = "SEO_seo_530"
SUPABASE_DB_PORT = "6543"

OPENAI_API_KEY = "...."
SUPABASE_URL = "...."
SUPABASE_KEY = "...."
client = OpenAI(api_key=OPENAI_API_KEY)
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)


# =====================================================
# --- Fetch microclusters ---
# =====================================================
def get_microclusters_for_cluster(cluster_id: int):
    response = supabase.table("cluster_micro").select(
        "microcluster_id, microcluster_name, representative_text, representative_keywords"
    ).eq("cluster_id", cluster_id).execute()

    if not response.data:
        print(f"⚠️ No microclusters found for cluster_id {cluster_id}")
        return []

    print(f"✅ Retrieved {len(response.data)} microclusters for cluster {cluster_id}")
    return response.data


# =====================================================
# --- Prompt builders ---
# =====================================================
def build_macro_prompt(cluster_id: int, microclusters: list, batch_id=None) -> str:
    parts = []
    for m in microclusters:
        name = m.get("microcluster_name", "")
        text = m.get("representative_text", "")
        keywords = m.get("representative_keywords", "")
        parts.append(
            f"- Microcluster {m['microcluster_id']} ({name}):\n"
            f"  Summary: {text}\n"
            f"  Keywords: {keywords}\n"
        )

    joined = "\n\n".join(parts)

    prompt = f"""
You are analyzing a macrocluster of documents.
This macrocluster contains several microclusters, each with a summary and keywords.

Cluster ID: {cluster_id}{f", Batch {batch_id}" if batch_id else ""}

Here are the microclusters:
---
{joined}
---

Please create a structured JSON object with the following fields:

{{
  "summary": "A 2–3 sentence summary of the main theme of this batch of microclusters.",
  "key_points": ["3–6 bullet points highlighting recurring themes."],
  "keywords": ["10–20 representative keywords or phrases (short, lowercase)."],
  "batch_name": "A short 2–5 word descriptive label for this batch."
}}

Rules:
- Do not invent facts not present in the microclusters.
- Use clear, concise English.
- Keywords should be actual recurring terms across microclusters.
- The batch_name should be broad but specific enough to describe the set.
""".strip()

    return prompt


def build_reduce_prompt(cluster_id: int, batch_results: list) -> str:
    parts = []
    for i, br in enumerate(batch_results, start=1):
        summary = br["llm_result"].get("summary", "")
        key_points = br["llm_result"].get("key_points", [])
        keywords = br["llm_result"].get("keywords", [])
        batch_name = br["llm_result"].get("batch_name", "")
        parts.append(
            f"- Batch {i} ({batch_name}):\n"
            f"  Summary: {summary}\n"
            f"  Key Points: {key_points}\n"
            f"  Keywords: {keywords}\n"
        )

    joined = "\n\n".join(parts)

    prompt = f"""
You are analyzing a macrocluster of documents.

Cluster ID: {cluster_id}

You are given summaries of several batches of microclusters.
Each batch has its own summary, key points, and keywords.

Here are the batch-level summaries:
---
{joined}
---

Please create a structured JSON object with the following fields:

{{
  "summary": "A 3–5 sentence integrated summary of the entire macrocluster.",
  "key_points": ["5–8 key points synthesizing the recurring ideas across batches."],
  "keywords": ["15–25 representative keywords or phrases (short, lowercase)."],
  "cluster_name": "A short 2–5 word descriptive label for the entire macrocluster."
}}

Rules:
- Focus on themes that recur across multiple batches.
- Do not repeat batch-specific details unless central to the overall cluster.
- Keywords should represent the macrocluster as a whole.
- The cluster_name should be broad but distinctive.
""".strip()

    return prompt


# =====================================================
# --- LLM Runner ---
# =====================================================
def run_llm(prompt: str) -> dict:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that outputs only valid JSON."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.3,
    )

    raw_text = response.choices[0].message.content.strip()

    try:
        return json.loads(raw_text)
    except Exception:
        print("⚠️ Invalid JSON from LLM, returning raw text.")
        return {"raw": raw_text}


# =====================================================
# --- Save final macrocluster result ---
# =====================================================
def save_macrocluster_result(cluster_id: int, final_result: dict):
    summary = final_result.get("summary", "")
    key_points = final_result.get("key_points", [])
    keywords = final_result.get("keywords", [])

    # Handle cluster_name correctly
    cluster_name = final_result.get("cluster_name")
    if not cluster_name:
        cluster_name = final_result.get("batch_name", f"Cluster {cluster_id}")  # ✅ fallback

    # Representative text
    representative_text = summary
    if isinstance(key_points, list) and key_points:
        representative_text += "\n\n- " + "\n- ".join(key_points)

    # Keywords format
    if isinstance(keywords, list):
        keywords_str = ", ".join(keywords)
    else:
        keywords_str = str(keywords)

    data = {
        "cluster_id": cluster_id,
        "cluster_name": cluster_name,  # always filled now ✅
        "representative_text": representative_text,
        "representative_keywords": keywords_str,
    }

    result = supabase.table("cluster_macro").upsert(data).execute()
    print("✅ Saved to Supabase:", result)


# =====================================================
# --- Macrocluster Summarization Pipeline ---
# =====================================================
def summarize_macrocluster(cluster_id: int, batch_size: int = 25):
    microclusters = get_microclusters_for_cluster(cluster_id)
    if not microclusters:
        return {}

    # Stage 1: split into batches
    n = len(microclusters)
    n_batches = math.ceil(n / batch_size)
    batch_results = []

    for i in range(n_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, n)
        batch_microclusters = microclusters[start:end]

        prompt_text = build_macro_prompt(cluster_id, batch_microclusters, batch_id=i+1)
        print("\n" + "=" * 60)
        print(f"⚡ Running LLM for {cluster_id}_batch{i+1} ({len(batch_microclusters)} microclusters)")
        llm_result = run_llm(prompt_text)
        batch_results.append({"prompt_id": f"{cluster_id}_batch{i+1}", "llm_result": llm_result})

    # Stage 2: reduce if multiple batches
    if len(batch_results) > 1:
        reduce_prompt = build_reduce_prompt(cluster_id, batch_results)
        print("\n⚡ Running LLM for FINAL REDUCE step")
        final_result = run_llm(reduce_prompt)
    else:
        final_result = batch_results[0]["llm_result"]

    # Save into Supabase
    save_macrocluster_result(cluster_id, final_result)

    return {
        "cluster_id": cluster_id,
        "batch_results": batch_results,
        "final_result": final_result
    }

In [ ]:
# =====================================================
# --- Example Run ---
# =====================================================
result = summarize_macrocluster(cluster_id=68, batch_size=13)
print("\n=== FINAL MACRO RESULT ===")
print(json.dumps(result["final_result"], indent=2))


✅ Retrieved 26 microclusters for cluster 68

⚡ Running LLM for 68_batch1 (13 microclusters)

⚡ Running LLM for 68_batch2 (13 microclusters)

⚡ Running LLM for FINAL REDUCE step
✅ Saved to Supabase: data=[{'cluster_id': 68, 'count': 171, 'cluster_name': 'Chevrolet Model Insights', 'representative_text': 'This macrocluster focuses on comprehensive reviews and specifications of various Chevrolet models, including popular options like the Camaro and Lacetti. It highlights user opinions, pricing, and features while providing insights into comparisons with similar vehicles. The inclusion of media elements such as photos and videos enhances the information for potential buyers and enthusiasts.\n\n- Emphasis on reviews and specifications for a range of Chevrolet models.\n- Focus on user opinions and detailed feature descriptions.\n- Inclusion of media content like photos and videos across documents.\n- Discussion of pricing information and insurance calculation services.\n- Comparisons with si